## 업무 자동화를 위한 쿠팡 파트너스 API 활용하기

### 0. 쿠팡파트너스 사전 준비
#### 0.1. API 이해하기
> 이미 slack API 에서 설명드렸으므로 간략하게

- 웹상에서 데이터를 주고 받는 방법: HTTP 프로토콜
- HTTP 프로토콜을 사용해서, 데이터를 요청하는 방법
  - GET, POST 등
  - requests 라이브러리를 사용해서, 데이터 요청 가능
    - GET: requests.get()
    - POST: reqeusts.post()
    
<center><img src="https://www.fun-coding.org/00_Images/web_http.png" height=350 /></center>

#### 0.1. 회원가입
#### 0.2. API 발급
<img src="https://www.fun-coding.org/00_Images/coupang_api_apply.png">
<img src="https://www.fun-coding.org/00_Images/coupang_api.png">

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">API 생성 버튼이 안보이실 경우</font><br>
쿠팡 파트너스 API 신청 방법이 매우 어렵게 바꼈습니다. <br>
API 생성버튼이 보이려면 쿠팡파트너스 링크로 구매가 한건 이상 이루어져야 하는것으로 바꼈다고 합니다. <br>
수시로 바뀌는 부분이라 영상으로는 한계가 있어보이고, 수업 노트/주피터 노트북 상으로 바뀔 때마다 업데이트하겠습니다.! <br>

간략히 설명드리면 <br>

1. 쿠팡 파트너스에서 내가 원하는 상품의 쿠팡 파트너스 링크를 직접 생성함 <br>
2. 해당 링크를 네이버 블로그등 기존에 활용 가능한 블로그에 넣고, 하단부에 '파트너스 활동을 통해 일정액의 수수료를 제공받을 수 있음' 문구를 넣음 <br>
3. 해당 상품을 직접 링크를 타고 쿠팡에 들어가서 구매 <br>

이렇게 한 후에야 API 생성버튼이 보인다고 합니다. <br>
이 부분 꼭 확인부탁드립니다.<br>

가장 쉽게 설명한 최신 유튜브를 참고하셔서 이 부분도 확인부탁드립니다.<br>

https://youtu.be/IO1lEvVOvDY</div>


- API 발급
<img src="https://www.fun-coding.org/00_Images/coupang_api_key.png">

> 혹시 안되시면 말씀해주시면, 함께 해결해보겠습니다. <br>
> API 키는 어디에도 유출하시면 안됩니다! 악의적인 해킹의 도구로 활용될 수 있으니까요.

### 1. 파이썬으로 쿠팡 파트너스 API 활용 준비
- 모든 API는 사용하라고 만든 것이므로, 사용 방법에 대해 나와 있습니다.
  - https://partners.coupang.com/#help/open-api
- 수시로 변할 수 있기 때문에, 가능한 가이드해주신 코드를 기반으로 사용하는 것이 가장 좋습니다.

### 1.1. 필요 라이브러리 임포트

In [6]:
import hmac
import hashlib
import binascii
import os
import time
import requests
import json
import urllib

### 1.2. authentication 정보 생성
- 참고: HMAC (Hash-based Message Authentication)
  - 해싱 기법을 적용하여 메시지의 위변조를 방지하는 기법
- 현재 시간, Secret Key를 활용하여 HMAC 기법을 사용해서 일종의 signature를 만들고,
  - Access Key 와 함께 쿠팡 파트너스 API 요청 헤더에 넣어서 위변조를 방지하고자 함

> 별도 템플릿 코드를 만들기보다, 가이드 문서대로 구현하는 것이 좋음 <br>
> 수시로 변경 가능성이 있기 때문임

In [7]:
from time import gmtime

def generateHmac(method, url, secretKey, accessKey):
    path, *query = url.split("?")
    os.environ["TZ"] = "GMT+0"
    datetime = time.strftime('%y%m%d')+'T'+time.strftime('%H%M%S')+'Z'
    message = datetime + method + path + (query[0] if query else "")

    signature = hmac.new(bytes(secretKey, "utf-8"),
                         message.encode("utf-8"),
                         hashlib.sha256).hexdigest()

    return "CEA algorithm=HmacSHA256, access-key={}, signed-date={}, signature={}".format(accessKey, datetime, signature)

#### 참고: 쿠팡파트너스 API 가 정상동작하지 않을 시 고려할 점
- 자신의 PC 시간이 한국시간으로 설정되어 있지 않은 경우, 정상 실행이 안될 수 있으므로, 자신의 PC 시간을 한국시간으로 설정하고, 실행해주세요!
- 한국설정으로 설정했는데도 안된다면, 셀을 만드신 후에, generateHmac 함수를 다음 코드로 선언한 후에 실행해보세요
  - 다음 코드는 별도로 셀을 만들고, 그 안에 다음 코드를 복사해서 넣으신 후에 실행하셔야 합니다.

```python
from time import gmtime

def generateHmac(method, url, secretKey, accessKey):
    path, *query = url.split("?")
    os.environ["TZ"] = "GMT+0"
    datetime = time.strftime('%y%m%d', gmtime())+'T'+time.strftime('%H%M%S', gmtime())+'Z'
    message = datetime + method + path + (query[0] if query else "")

    signature = hmac.new(bytes(secretKey, "utf-8"),
                         message.encode("utf-8"),
                         hashlib.sha256).hexdigest()

    return "CEA algorithm=HmacSHA256, access-key={}, signed-date={}, signature={}".format(accessKey, datetime, signature)
```

### 2. 키워드 기반 상위 30개 검색 상품의 회원 트래킹 코드가 포함된 링크 가져오기 API

### 2.1. secret key 와 access key 적용

#### secret key 와 access key 데이터 숨기기
> 파이썬 코드 상에 자신의 계정과 암호를 기입하지 않고, 계정과 암호 데이터를 사용할 수 있는 기법

- class101_autopython 폴더 안에 
   - 텍스트 에디터로 settings.py 파일에, COUPANG_SECRET_KEY 와 COUPANG_ACCESS_KEY 수정
   
    ```python
    COUPANG_SECRET_KEY = '자신의 쿠팡 파트너스 secret key'
    COUPANG_ACCESS_KEY = '자신의 쿠팡 파트너스 access key'
    ```

```python 
# 노출 방지를 위해서 마크다운 형태로 셀을 바꿨습니다. 코드만 참고하셔서 실행해보세요 
import settings

print (settings.COUPANG_SECRET_KEY, settings.COUPANG_ACCESS_KEY)
```

In [8]:
import settings

access_key = settings.COUPANG_ACCESS_KEY # API access key
secret_key = settings.COUPANG_SECRET_KEY # API secret key

### 2.2. 검색 API 와 필요 인자 선언

In [9]:
request_method = "GET"

- 검색 API 는 크게 키워드와 검색 랭킹 기반, 몇 개까지 가져올지를 정의해야 함

In [10]:
keyword = '선물'
limit = 30

In [11]:
partners_url = "/v2/providers/affiliate_open_api/apis/openapi/products/search?keyword=" 
partners_url += urllib.parse.quote(keyword) + "&limit=" + str(limit)

### 2.3. authorization 정보 생성

In [12]:
authorization = generateHmac(request_method, partners_url, secret_key, access_key)

In [13]:
authorization

'CEA algorithm=HmacSHA256, access-key=0853fd8f-0f2b-45f9-8792-cdaae7e221a9, signed-date=201114T015806Z, signature=10f30bbe3e2fa483517f4b3a831a38f4124d2bf70abde5933a69d5af3d04b452'

### 2.4. 검색 API 호출을 위한 필요 인자 선언

In [14]:
request_domain = "https://api-gateway.coupang.com"
request_data = ''

### 2.5. 템플릿 코드13: 쿠팡 파트너스 API 호출 결과 가져오기

In [15]:
# 가이드 문서대로 authorization 정보는 별도 함수로 그대로 선언합니다.
def generateHmac(method, url, secretKey, accessKey):
    path, *query = url.split("?")
    os.environ["TZ"] = "GMT+0"
    datetime = time.strftime('%y%m%d')+'T'+time.strftime('%H%M%S')+'Z'
    message = datetime + method + path + (query[0] if query else "")

    signature = hmac.new(bytes(secretKey, "utf-8"),
                         message.encode("utf-8"),
                         hashlib.sha256).hexdigest()

    return "CEA algorithm=HmacSHA256, access-key={}, signed-date={}, signature={}".format(accessKey, datetime, signature)

# 가이드 문서대로 내부 코드를 적되, 다양한 API에 대응 가능토록 일부만 변경
def coupang_partners_template(partners_domain, partners_api, partners_method, authorization, request_data):
    request_url = "{}{}".format(partners_domain, partners_api)
    response = requests.request(method=partners_method, url=request_url,
                                headers={
                                    "Authorization": authorization,
                                    "Content-Type": "application/json"
                                },
                                data=json.dumps(request_data)
                                )
    return response.json()

### 2.6. 검색 API 호출
> 주의: 하루 호출량이 매우 제한적임 (한번만 테스트해보세요!)

In [16]:
data_json = coupang_partners_template(request_domain, partners_url, request_method, authorization, request_data)

In [17]:
data_json

{'rCode': '0',
 'rMessage': '게시글 작성 시, "파트너스 활동을 통해 일정액의 수수료를 제공받을 수 있음"을 기재하셔야 합니다',
 'data': {'landingUrl': 'https://link.coupang.com/re/AFFSRP?lptag=AF8181387&pageKey=%EC%84%A0%EB%AC%BC&traceid=V0-163-523d7acba7b43632',
  'productData': [{'productId': 1581951,
    'productName': '신라명과 셀레브르쿠키 + 쇼핑백',
    'productPrice': 14670,
    'productImage': 'https://static.coupangcdn.com/image/retail/images/409157280392964-e455c3cd-9e57-4d5c-b2d2-a19687c53f79.jpg',
    'productUrl': 'https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=1581951&itemId=6912360&vendorItemId=3009171682&traceid=V0-153-5bec7c22f4aca8eb',
    'keyword': '선물',
    'rank': 1,
    'isRocket': True,
    'isFreeShipping': False},
   {'productId': 255921241,
    'productName': '메디플라워 보니타 가든 핸드크림 6종세트 + 쇼핑백, 2세트',
    'productPrice': 15900,
    'productImage': 'https://static.coupangcdn.com/image/product/image/vendoritem/2019/09/19/5041350690/5c63fa3f-43b6-499b-b440-0002ecffa0ac.jpg',
    'productUrl': 'https://link.co

### 2.6. json 데이터 확인하기

- pprint.pprint() 함수를 사용해서, 좀더 명확하게 json 데이터 확인하기
- productUrl 에 있는 링크를 통해 들어가서, 상품 구매시 일정 수수료를 받을 수 있음

> 관련 게시글 작성 시, "파트너스 활동을 통해 일정액의 수수료를 제공받을 수 있음"을 기재해야 함

```json
{
    'isRocket': True,
    'keyword': '떡볶이',
    'productId': 188709954,
    'productImage': 'https://static.coupangcdn.com/image/retail/images/261952026077622-faeeea6c-f519-4442-b000-fad3bf0c0089.jpg',
    'productName': '곰곰 떡볶이 옛날맛 (냉동), 560g, 3개',
    'productPrice': 12530,
    'productUrl': 'https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=188709954&itemId=538914812&vendorItemId=5016667484&traceid=V0-153-ad7a6228f69fa5ea',
    'rank': 3
}
```

- 상품 데이터는 결국 productData 필드에 있음

In [42]:
datas1 = data_json['data']['productData']
for item in datas1:
    print (item['productName'], item['productPrice'], item['productUrl'])

제로캔들 소이캔들 대용량 향초 280ml, 레몬라벤더, 1개 12900 https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=210587431&itemId=537334187&vendorItemId=4403068224&traceid=V0-153-f2d59b53f2ad4fab
웰던커피 더치커피 선물세트, 브라질 산토스 + 콜롬비아 슈프리모 + 에디오피아 예가체프, 1세트 16200 https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=1391908041&itemId=2427296415&vendorItemId=70421296525&traceid=V0-153-66b3f5d596b9f2e9
투데이넛 너트한줌 프리미엄 견과 선물세트 30개입, 600g, 1세트 15390 https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=158935&itemId=295153&vendorItemId=3000205575&traceid=V0-153-75b0f0bd005a69e4
라돌체비타 프리미엄 허브차 9종 선물세트 + 쇼핑백, 허브티 9종, 1세트 19500 https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=169551284&itemId=485080740&vendorItemId=4222394102&traceid=V0-153-53682ebca42b2943
제로캔들 소이 캔들 대용량향초 660ml, 블랙체리, 1개 18900 https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=210587460&itemId=537334150&vendorItemId=4403068151&traceid=V0-153-f798c223c506e6e2
제로캔들 소이캔들 대용량 향초 1000ml, 블랙체리, 1개 27890 https://link.coupang.

### 3. 쿠팡 URL을 회원 트래킹 코드가 포함된 링크로 변환하는 API

In [43]:
request_method = "POST"
partners_url = "/v2/providers/affiliate_open_api/apis/openapi/v1/deeplink" 
request_domain = "https://api-gateway.coupang.com"
request_data = { "coupangUrls": [
    "https://www.coupang.com/vp/products/233398511?itemId=741212717&vendorItemId=4871712111&q=%EB%AA%A8%EB%8B%88%ED%84%B0&itemsCount=36&searchId=55fe11377e9f402486b1e702cbc52b12&rank=9&isAddedCart="
]}

In [44]:
authorization = generateHmac(request_method, partners_url, secret_key, access_key)

In [49]:
data_json = coupang_partners_template(request_domain, partners_url, request_method, authorization, request_data)

In [50]:
data_json

{'rCode': '0',
 'rMessage': '',
 'data': [{'originalUrl': 'https://www.coupang.com/vp/products/233398511?itemId=741212717&vendorItemId=4871712111&q=%EB%AA%A8%EB%8B%88%ED%84%B0&itemsCount=36&searchId=55fe11377e9f402486b1e702cbc52b12&rank=9&isAddedCart=',
   'shortenUrl': 'https://coupa.ng/bDbf70',
   'landingUrl': 'https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=233398511&itemId=741212717&vendorItemId=4871712111&traceid=V0-183-5397e9194e245476'}]}

In [52]:
datas1 = data_json['data']
for item in datas1:
    print (item['landingUrl'])

https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=233398511&itemId=741212717&vendorItemId=4871712111&traceid=V0-183-5397e9194e245476


### 4. 관련 상품 상세 정보 가져오기 (템플릿 코드14)
- selenium 을 사용해서 관련 상품 상세 정보를 가져올 수 있음
- 상세 정보는 모두 이미지이기 때문에 이미지를 다운로드 받아야 함
- selenium 템플릿 코드를 기반으로, 사이트 관련 코드를 추가함

> 참고: selenium 도 있지만, headless chrome 기술도 있음 <br>
> 웹브라우저를 안띄우지만, 크롤링으로 접근했음을 체크하기 쉬움

In [73]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 

def get_details_product_template(url):
    # 각 PC 환경에 맞게 수정 필요
    driver_location = './chromedriver'
    css_selector_title = 'h2.prod-buy-header__title'
    css_selector_price = 'div.prod-coupon-price.prod-major-price > span.total-price > strong'
    css_selector_ori_price = 'div.prod-sale-price > span.total-price > strong'    
    css_selector_img = 'div.product-detail-content-inside div.subType-IMAGE > img'
    img_path = '/Users/davelee/class101_autopython/00_coupang_product_imgs'
    vendoritemid = url.split('=')[4].split('&')[0]
    
    return_data = list()
    return_data.append(vendoritemid)
    
    driver = webdriver.Chrome(driver_location)
    driver.get(url)
    time.sleep(3)

    titles = driver.find_elements_by_css_selector(css_selector_title)
    for item in titles:
        return_data.append(item.text)

    prices = driver.find_elements_by_css_selector(css_selector_price)
    for item in prices:
        if item.text == '':
            prices = driver.find_elements_by_css_selector(css_selector_ori_price)
            for item2 in prices:
                return_data.append(item2.text)
        else:
            return_data.append(item.text)

    return_data.append(url)
    imgs = driver.find_elements_by_css_selector(css_selector_img)
    images = list()
    for num, item in enumerate(imgs):
        src = item.get_attribute('src')
        '''
        if 'vendor_inventory' not in src and ('thumbnail' not in src or 'remote' not in src):
            continue
        filename = vendoritemid + '-' + str(num) + ".png"  # ventoritemid-이미지번호.png 파일을 생성
        full_filename = img_path + "/" + filename
        urllib.request.urlretrieve(src, full_filename) # 해당 이미지 파일을 지정 폴더에 다운로드
        images.append(('image', (filename, open(full_filename, 'rb'), 'image/png')))
        return_data.append(images)
        '''
        return_data.append(src)
        

    driver.quit()
    return return_data

In [74]:
get_details_product_template(' https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=118610786&itemId=353942329&vendorItemId=3862448541&traceid=V0-153-56568432a6ae8bdc')

['3862448541',
 '빅토리아 그림이 있는 차 선물세트 6호 + 쇼핑백',
 '9,900원',
 ' https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=118610786&itemId=353942329&vendorItemId=3862448541&traceid=V0-153-56568432a6ae8bdc',
 [('image',
   ('3862448541-0.png',
    <_io.BufferedReader name='/Users/davelee/class101_autopython/00_coupang_product_imgs/3862448541-0.png'>,
    'image/png'))]]

### (활용) 템플릿 코드6: 리스트 변수 데이터를 엑셀 파일로 만들기

In [75]:
import openpyxl

def write_excel_template(filename, sheetname, listdata):
    excel_file = openpyxl.Workbook()
    excel_sheet = excel_file.active

    # 엑셀 이쁘게 하기에서 셀 사이즈, 타이틀행만 추가
    excel_sheet.column_dimensions['A'].width = 20
    excel_sheet.column_dimensions['B'].width = 80
    excel_sheet.column_dimensions['D'].width = 200    
    excel_sheet.append(['ventoritemid', '상품 타이틀', '가격', '링크'])
    
    if sheetname != '':
        excel_sheet.title = sheetname
    
    for item in listdata:
        excel_sheet.append(item)
    excel_file.save(filename)
    excel_file.close()

### 오늘 날짜 가져오기
- datetime 라이브러리의 datetime.date.today() 함수를 사용하면 됨
  - 단 날짜를 나타내는 특별한 데이터 포멧이 있음
  - strftime() 함수로 날짜를 다양한 포멧에 맞게, 문자열로 변환할 수 있음
    - 예: %Y-%m-%d
  
| 표시 | 의미                                |
|------|:-------------------------------------|
| %y   | 연도를 축약해서 표시, 예: 21        |
| %Y   | 연도를 축약하지 않고 표시, 예: 2021 |
| %m   | 월을 두자릿 수로 표시, 예: 01 ~ 12  |
| %-m  | 월을 0 없이 표시, 예: 1 ~ 12        |
| %d   | 일을 두자릿 수로 표시, 예: 01 ~ 31  |
| %-d  | 일을 0 없이 표시, 예: 1 ~ 31        |

In [76]:
import datetime
data1 = datetime.date.today()
print (type(data1), data1)

<class 'datetime.date'> 2020-05-27


In [77]:
import datetime
data1 = datetime.date.today()
data1 = data1.strftime('%Y-%m-%d')
print (type(data1), data1)

<class 'str'> 2020-05-27


In [78]:
import datetime
data1 = datetime.date.today()
data1 = data1.strftime('%Y-%m-%d')

In [79]:
listdata = [['4871712111', 'LG전자 86.7cm WFHD 울트라와이드 HDR 모니터', '383,000원']]

In [80]:
write_excel_template('tmp.xlsx', data1, listdata)

### 5. 원하는 상품 가져와서, 상품 타이틀, 가격, 상품 이미지를 엑셀로 가져오기 
> 업무 자동화 목표에 맞추어, 어느 작업을 하시든 단순 업무를 자동화하기 위한 예를 보여드립니다. <br>
> 업무 자동화라기보다 각자 원하시는 수익화 프로그램을 원하시는 경우에는 <br>
> 1:1 코칭을 통해 빠르게 구현 및 실현 가능하실 수 있도록 구현 범위, 기능 조정, 구현 방법에 대해 알려드립니다.

### 5.1. 예: 카테고리 베스트 상품 가져오기

In [81]:
request_method = "GET"
partners_url = "/v2/providers/affiliate_open_api/apis/openapi/products/bestcategories/1001" 
request_domain = "https://api-gateway.coupang.com"
request_data = ''

In [82]:
authorization = generateHmac(request_method, partners_url, secret_key, access_key)
data_json = coupang_partners_template(request_domain, partners_url, request_method, authorization, request_data)

### 필요한 문법
- json 과 사전(dictionary) 데이터 구조는 동일하게 다룰 수 있음 (일전 챕터에서 설명)
- 사전에서 각 키 값 가져오기 사전변수.keys() : 리스트로 키 값을 가져옴


In [91]:
for item in data_json['data'][0].keys():
    print (item)

productId
productName
productPrice
productImage
productUrl
categoryName
keyword
rank
isRocket


### (활용) 템플릿 코드6: 리스트 변수 데이터를 엑셀 파일로 만들기

In [92]:
import openpyxl

def write_excel_template(filename, sheetname, listdata):
    excel_file = openpyxl.Workbook()
    excel_sheet = excel_file.active

    # 엑셀 이쁘게 하기에서 셀 사이즈, 타이틀행만 추가
    excel_sheet.column_dimensions['A'].width = 20
    excel_sheet.column_dimensions['B'].width = 70
    excel_sheet.column_dimensions['C'].width = 10    
    excel_sheet.column_dimensions['D'].width = 120    
    excel_sheet.column_dimensions['E'].width = 120        
    excel_sheet.append(['productId', 'productName', 'productPrice', 'productImage', 'productUrl', 'categoryName', 'keyword', 'rank', 'isRocket'])
    
    if sheetname != '':
        excel_sheet.title = sheetname
    
    for item in listdata:
        excel_sheet.append(item)
    excel_file.save(filename)
    excel_file.close()

In [93]:
data_json['data']

[{'productId': 185502198,
  'productName': '쿠팡 브랜드 - 베이스알파 에센셜 남녀공용 30수 라운드 반팔티 화이트 3p',
  'productPrice': 12500,
  'productImage': 'https://static.coupangcdn.com/image/retail/images/7028390930032-016cbfa5-5fa7-4b65-aba1-cd7d2d54861b.jpg',
  'productUrl': 'https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=185502198&itemId=530600312&vendorItemId=4381872911&traceid=V0-113-d1b4e08408552d8e',
  'categoryName': '여성패션',
  'keyword': '여성패션',
  'rank': 1,
  'isRocket': True},
 {'productId': 242527845,
  'productName': '엘레스 여성반팔티 당일발송 빅사이즈 자체제작 반팔 티셔츠',
  'productPrice': 9500,
  'productImage': 'https://static.coupangcdn.com/image/vendor_inventory/3907/0700399e291d57b162c93624d036d5e4fa304f3b49bb63b20d0f428b0224.jpg',
  'productUrl': 'https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=242527845&itemId=770221714&vendorItemId=4944830232&traceid=V0-113-1b2fda6b52d4e5dd',
  'categoryName': '여성패션',
  'keyword': '여성패션',
  'rank': 2,
  'isRocket': False},
 {'productId': 1419894910,
  

In [94]:
listdata = list()
for item in data_json['data']:
    itemdata = list()
    for item2 in item.keys():
        itemdata.append(item[item2])
    listdata.append(itemdata)

In [95]:
listdata

[[185502198,
  '쿠팡 브랜드 - 베이스알파 에센셜 남녀공용 30수 라운드 반팔티 화이트 3p',
  12500,
  'https://static.coupangcdn.com/image/retail/images/7028390930032-016cbfa5-5fa7-4b65-aba1-cd7d2d54861b.jpg',
  'https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=185502198&itemId=530600312&vendorItemId=4381872911&traceid=V0-113-d1b4e08408552d8e',
  '여성패션',
  '여성패션',
  1,
  True],
 [242527845,
  '엘레스 여성반팔티 당일발송 빅사이즈 자체제작 반팔 티셔츠',
  9500,
  'https://static.coupangcdn.com/image/vendor_inventory/3907/0700399e291d57b162c93624d036d5e4fa304f3b49bb63b20d0f428b0224.jpg',
  'https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=242527845&itemId=770221714&vendorItemId=4944830232&traceid=V0-113-1b2fda6b52d4e5dd',
  '여성패션',
  '여성패션',
  2,
  False],
 [1419894910,
  '티데일리 남여공용 이프나우 오버핏 반팔 라운드 티셔츠 20수 (4color) 반소매',
  8900,
  'https://static.coupangcdn.com/image/vendor_inventory/6b4b/a763cf60972d57519149567a6935cd2728b6620602803b7b9ebf95048746.jpg',
  'https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=1419894

In [98]:
import datetime
data1 = datetime.date.today()
data1 = data1.strftime('%Y-%m-%d')

In [99]:
write_excel_template('tmp.xlsx', data1, listdata)

### 5.2. 예
1. 내가 선택한 상품 엑셀에 넣고, 
2. 엑셀 파일 읽어서, 
3. 쿠팡 파트너스 API 와 selenium 으로,
4. 딥링크와 원하는 상품 정보 가져와서,
5. 별도 엑셀 파일로 저장하기

In [131]:
request_method = "POST"
partners_url = "/v2/providers/affiliate_open_api/apis/openapi/v1/deeplink" 
request_domain = "https://api-gateway.coupang.com"
request_data = { "coupangUrls": []}

### (활용) 템플릿 코드5: 엑셀 파일 읽어서 리스트 변수로 가져오기

In [128]:
import openpyxl

def read_excel_template(filename, sheetname):
    excel_file = openpyxl.load_workbook(filename)
    
    if sheetname == '':
        excel_sheet = excel_file.active
    else:
        excel_sheet = excel_file[sheetname]

    return_data = list()    
    for item in excel_sheet.rows:
        datas1 = list()
        for item2 in item:
            datas1.append(item2.value)
        return_data.append(datas1)
        
    excel_file.close()
    return return_data

In [129]:
datas1 = read_excel_template('coupang_list.xlsx', '2020-05-20')

In [130]:
datas1

[['https://www.coupang.com/vp/products/327360263?itemId=1047441230&vendorItemId=5510912600&sourceType=CAMPAIGN&campaignId=5173&categoryId=0&isAddedCart='],
 ['https://www.coupang.com/vp/products/189267000?itemId=540447825&vendorItemId=5017244964&sourceType=CAMPAIGN&campaignId=5173&categoryId=0&isAddedCart='],
 ['https://www.coupang.com/vp/products/1194639177?itemId=2177439743&vendorItemId=70175514691&sourceType=CAMPAIGN&campaignId=5173&categoryId=0&isAddedCart='],
 ['https://www.coupang.com/vp/products/192159873?itemId=549746692&vendorItemId=4442528641&sourceType=CAMPAIGN&campaignId=5173&categoryId=0&isAddedCart='],
 ['https://www.coupang.com/vp/products/346765263?itemId=1100652101&vendorItemId=5628102991&sourceType=CAMPAIGN&campaignId=5173&categoryId=0&isAddedCart=']]

In [132]:
for item in datas1:
    request_data['coupangUrls'].append(item[0])

In [133]:
request_data

{'coupangUrls': ['https://www.coupang.com/vp/products/327360263?itemId=1047441230&vendorItemId=5510912600&sourceType=CAMPAIGN&campaignId=5173&categoryId=0&isAddedCart=',
  'https://www.coupang.com/vp/products/189267000?itemId=540447825&vendorItemId=5017244964&sourceType=CAMPAIGN&campaignId=5173&categoryId=0&isAddedCart=',
  'https://www.coupang.com/vp/products/1194639177?itemId=2177439743&vendorItemId=70175514691&sourceType=CAMPAIGN&campaignId=5173&categoryId=0&isAddedCart=',
  'https://www.coupang.com/vp/products/192159873?itemId=549746692&vendorItemId=4442528641&sourceType=CAMPAIGN&campaignId=5173&categoryId=0&isAddedCart=',
  'https://www.coupang.com/vp/products/346765263?itemId=1100652101&vendorItemId=5628102991&sourceType=CAMPAIGN&campaignId=5173&categoryId=0&isAddedCart=']}

### 템플릿 코드15
1. 원하는 상품 링크를 엑셀로부터 가져와서,
2. 나의 트래킹 코드가 포함된 링크로 가져오고,
3. 상품 타이틀과 가격, 상품 첫번째 상세 이미지를 가져와서,
4. 트래킹 코드가 포함된 링크, 상품 타이틀, 가격를 converted-파일명 으로 저장

In [135]:
def get_mylink_coupang_partner_template(filename, sheetname):
    excel_data = list()
    request_data = { "coupangUrls": []}
    datas1 = read_excel_template(filename, sheetname)
    
    for item in datas1:
        request_data['coupangUrls'].append(item[0])
    authorization = generateHmac(request_method, partners_url, secret_key, access_key)
    data_json = coupang_partners_template(request_domain, partners_url, request_method, authorization, request_data)
    datas1 = data_json['data']
    
    for item in datas1:
        datas2 = get_details_product_template(item['landingUrl']) # 'originalUrl'과 'landingUrl' 가격이 다를 수 있음! (원하는 필드로 변경)
        excel_data.append(datas2)
    
    write_excel_template('converted_' + filename, sheetname, excel_data)
    return excel_data

### 원하는 나만의 작업을 위해 템플릿 코드 활용하기

#### (활용) 관련 상품 상세 정보 가져오기 (템플릿 코드14)

In [140]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 

def get_details_product_template(url):
    # 각 PC 환경에 맞게 수정 필요
    driver_location = './chromedriver'
    css_selector_title = 'h2.prod-buy-header__title'
    css_selector_price = 'div.prod-coupon-price.prod-major-price > span.total-price > strong'
    css_selector_ori_price = 'div.prod-sale-price > span.total-price > strong'    
    css_selector_img = 'div.product-detail-content-inside div.subType-IMAGE > img'
    img_path = '/Users/davelee/class101_autopython/00_coupang_product_imgs'
    vendoritemid = url.split('=')[4].split('&')[0]
    
    return_data = list()
    return_data.append(vendoritemid)
    
    driver = webdriver.Chrome(driver_location)
    driver.get(url)
    time.sleep(3)

    titles = driver.find_elements_by_css_selector(css_selector_title)
    for item in titles:
        return_data.append(item.text)

    prices = driver.find_elements_by_css_selector(css_selector_price)
    for item in prices:
        if item.text == '':
            prices = driver.find_elements_by_css_selector(css_selector_ori_price)
            for item2 in prices:
                return_data.append(item2.text)
        else:
            return_data.append(item.text)

    return_data.append(url)
    img = driver.find_element_by_css_selector(css_selector_img)
    return_data.append(img.get_attribute('src'))

    driver.quit()
    return return_data

#### (활용) 템플릿 코드6: 리스트 변수 데이터를 엑셀 파일로 만들기

In [141]:
import openpyxl

def write_excel_template(filename, sheetname, listdata):
    excel_file = openpyxl.Workbook()
    excel_sheet = excel_file.active

    # 엑셀 이쁘게 하기에서 셀 사이즈, 타이틀행만 추가
    excel_sheet.column_dimensions['A'].width = 20
    excel_sheet.column_dimensions['B'].width = 70
    excel_sheet.column_dimensions['C'].width = 10    
    excel_sheet.column_dimensions['D'].width = 120    
    excel_sheet.column_dimensions['E'].width = 120        
    excel_sheet.append(['productId', 'productName', 'productPrice', 'productUrl', 'productDetailImage'])
    
    if sheetname != '':
        excel_sheet.title = sheetname
    
    for item in listdata:
        excel_sheet.append(item)
    excel_file.save(filename)
    excel_file.close()

In [142]:
request_method = "POST"
partners_url = "/v2/providers/affiliate_open_api/apis/openapi/v1/deeplink" 
request_domain = "https://api-gateway.coupang.com"

In [143]:
datas1 = get_mylink_coupang_partner_template('coupang_list.xlsx', '2020-05-20')

In [139]:
datas1

[['5510912600',
  '곰곰 건강한 닭 도시락 시즌2 (냉동)',
  '18,810원',
  'https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=327360263&itemId=1047441230&vendorItemId=5510912600&traceid=V0-183-3e69348b296ebeed',
  'https://thumbnail10.coupangcdn.com/thumbnails/remote/q89/image/retail/images/626532702988649-b39d7a2f-06f7-47e7-b326-e478308370e2.jpg'],
 ['5017244964',
  '곰곰 건강한 밥 도시락 (냉동)',
  '18,810원',
  'https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=189267000&itemId=540447825&vendorItemId=5017244964&traceid=V0-183-c2fc81ad68c97e32',
  'https://thumbnail8.coupangcdn.com/thumbnails/remote/q89/image/retail/images/628624833311431-1261607c-05b5-473b-9d28-61fb7a2536de.jpg'],
 ['70175514691',
  '곰곰 반숙란 (냉장)',
  '7,960원',
  'https://link.coupang.com/re/AFFSDP?lptag=AF8181387&pageKey=1194639177&itemId=2177439743&vendorItemId=70175514691&traceid=V0-183-597e3f51b07d091b',
  'https://thumbnail10.coupangcdn.com/thumbnails/remote/q89/image/retail/images/859725361264848-e3aabb5c-6b4a-44e6-a8a7-50

### 6. (참고) 나만의 블로그로 쿠팡 파트너스 상품 포스팅하기
- 이 부분은 참고로 마지막 챕터에서 블로그 생성 방법과 함께 몰아서 설명드리겠습니다.